In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns 
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
from scipy.io import loadmat
from scipy import integrate
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy import integrate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sympy import *

In [ ]:
# single step
# time points
t = np.linspace(0, 100, num=100)
K = 10000
# N is total population
S = 9000
p = 900
m = 100
rp = 0
rm = 0
Gs = 0.0
D = 0.25
up = (3e-4)
um = up * 2
A = 1
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4])/K) - D)
  dpdt = spr[1] * (  
      ((n/A * fR + (A-n)/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4])/K)) 
      - D - up
      )
  dmdt = spr[2] * (  
      ((n/A * fR + (A-n)/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4])/K)) 
      - D - um
      )

  drpdt = spr[3] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4])/K)) - D) + (spr[1] * up)
  drmdt = spr[4] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4])/K)) - D) + (spr[2] * um)
  dspmdt = [dsdt, dpdt, dmdt, drpdt, drmdt]#, dpdt, dmdt, drdt]
  return dspmdt

# initial conditions
spr0 = (S, p, m, rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df1 = pd.DataFrame(s_p_r, columns=['Sensitive', 'Persister', 'Mutator', 'Resistant Persister', 'Resistant Mutator'])
df1['Time'] = range(0, 100)
df1['Total Cells'] = (df1['Sensitive'] + df1['Persister'] + df1['Mutator'] + df1['Resistant Persister'] + df1['Resistant Mutator'])
df1['Resistant'] = (df1['Resistant Persister'] + df1['Resistant Mutator'])
df1['Sensitive Fraction'] = df1['Sensitive']/df1['Total Cells']
df1['Persister Fraction'] = (df1['Persister'] + df1['Resistant Persister'])/df1['Total Cells']
df1['Mutator Fraction'] = (df1['Mutator'] + df1['Resistant Mutator'])/df1['Total Cells']
df1['Resistant Fraction'] = df1['Resistant']/df1['Total Cells']


In [ ]:
# two step
# time points
t = np.linspace(0, 100, num=100)
K = 10000
# N is total population
S = 9000
p1 = 900
m1 = 100
p2 = 0
m2 = 0
rp = 0
rm = 0
Gs = 0.0
D = 0.20
up = (3e-3) * 1
um = up * 2
A = 1
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K) - D)
  dp1dt = spr[1] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) 
      - D - up
      )
  dm1dt = spr[2] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) 
      - D - um
      )
  dp2dt = spr[3] * (  
      ((n/A * fR + (A-n)/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) 
      - D - up
      ) + (spr[1] * up)
  dm2dt = spr[4] * (  
      ((n/A * fR + (A-n)/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) 
      - D - um
      ) + (spr[2] * up)
  drpdt = spr[5] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) - D) + (spr[3] * up)
  drmdt = spr[6] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] +spr[5] + spr[6])/K)) - D) + (spr[4] * um)
  dspmdt = [dsdt, dp1dt, dm1dt, dp2dt, dm2dt, drpdt, drmdt]#, dpdt, dmdt, drdt]
  return dspmdt

# initial conditions
spr0 = (S, p1, m1, p2, m2, rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df2 = pd.DataFrame(s_p_r, columns=['Sensitive', 'Persister1', 'Mutator1', 'Persister2', 'Mutator2', 'Resistant Persister', 'Resistant Mutator'])
df2['Time'] = range(0, 100)
df2['Total Cells'] = (df2['Sensitive'] + 
                    df2['Persister1'] + df2['Mutator1'] + 
                    df2['Persister2'] + df2['Mutator2'] +
                    df2['Resistant Persister'] + df2['Resistant Mutator'])
df2['Resistant'] = (df2['Resistant Persister'] + df2['Resistant Mutator'])
df2['Sensitive Fraction'] = df2['Sensitive']/df2['Total Cells']
df2['Persister Fraction'] = (df2['Persister1'] + df2['Persister2'] + df2['Resistant Persister'])/df2['Total Cells']
df2['Mutator Fraction'] = (df2['Mutator1'] + df2['Mutator2'] + df2['Resistant Mutator'])/df2['Total Cells']
df2['Resistant Fraction'] = df2['Resistant']/df2['Total Cells']

In [ ]:
# Five Steps
# time points
t = np.linspace(0, 100, num = 100)
K = 10000
# N is total population
S = 9000
p1 = 900
m1 = 100
p2 = 0
m2 = 0
m3 = 0
p3 = 0
m4 = 0
p4 = 0
m5 = 0
p5 = 0
r = 0
Gs = 0.0
D = 0.25
up = (3e-3) * 4
um = up * 2
A = 4
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K) - D)
  dp1dt = spr[1] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - up
      )
  dm1dt = spr[2] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - um
      )
  dp2dt = spr[3] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - up) + (spr[1] * up)
      
  dm2dt = spr[4] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - um) + (spr[2] * um)

  dp3dt = spr[5] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - up) + (spr[3] * up)
      
  dm3dt = spr[6] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - um) + (spr[4] * um)

  dp4dt = spr[7] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - up) + (spr[5] * up)

  dm4dt = spr[8] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) 
      - D - um) + (spr[6] * um)
    
  drpdt = spr[9] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) - D) + (spr[7] * up)
    
  drmdt = spr[10] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10])/K)) - D) + (spr[8] * um)
    
  dspmdt = [dsdt, dp1dt, dm1dt, dp2dt, dm2dt, dp3dt, dm3dt, dp4dt, dm4dt, drpdt, drmdt]
  return dspmdt

# initial conditions
spr0 = (S, p1, m1, p2, m2, p3, m3, p4, m4, rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df5 = pd.DataFrame(s_p_r, columns=['Sensitive', 
                                  'Persister1', 'Mutator1', 
                                  'Persister2', 'Mutator2', 
                                  'Persister3', 'Mutator3', 
                                  'Persister4', 'Mutator4', 
                                  'Resistant Persister', 'Resistant Mutator'])
df5['Time'] = range(0, 100)
df5['Total Cells'] = (df5['Sensitive'] + 
                     df5['Persister1'] + df5['Mutator1'] + 
                     df5['Persister2'] + df5['Mutator2'] + 
                     df5['Persister3'] + df5['Mutator3'] + 
                     df5['Persister4'] + df5['Mutator4'] + 
                     df5['Resistant Persister'] + df5['Resistant Mutator'])
df5['Resistant'] = (df5['Resistant Persister'] + df5['Resistant Mutator'])
df5['Sensitive Fraction'] = df5['Sensitive']/df5['Total Cells']

df5['Persister Fraction'] = (df5['Persister1'] + df5['Persister2'] + 
                            df5['Persister3']+ df5['Persister4']
                            + df5['Resistant Persister'])/df5['Total Cells']
df5['Mutator Fraction'] = (df5['Mutator1'] + df5['Mutator2'] + 
                          df5['Mutator3'] + df5['Mutator4'] +
                          df5['Resistant Mutator'])/df5['Total Cells']
df5['Resistant Fraction'] = df5['Resistant']/df5['Total Cells']

In [ ]:
# Ten steps
# time points
t = np.linspace(0, 100, num = 100)
K = 10000
# N is total population
S = 9000
p1 = 900
m1 = 100
p2 = 0
m2 = 0
p3 = 0
m3 = 0
p4 = 0
m4 = 0
p5 = 0
m5 = 0
p6 = 0
m6 = 0
p7 = 0
m7 = 0
p8 = 0
m8 = 0
p9 = 0
m9 = 0
rp = 0
rm = 0
Gs = 0.0
D = 0.25
up = (3e-3) * 9
um = up * 2
A = 9
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K) - D)
  dp1dt = spr[1] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up
      )
  dm1dt = spr[2] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um
      )
  dp2dt = spr[3] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[1] * up)
      
  dm2dt = spr[4] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[2] * um)

  dp3dt = spr[5] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[3] * up)
      
  dm3dt = spr[6] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[4] * um)

  dp4dt = spr[7] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[5] * up)

  dm4dt = spr[8] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[6] * up)
      
  dp5dt = spr[9] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[7] * up)
      
  dm5dt = spr[10] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[8] * um)

  dp6dt = spr[11] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[9] * up)

  dm6dt = spr[12] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[10] * um)

  dp7dt = spr[13] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[11] * up)
      
  dm7dt = spr[14] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[12] * um)

  dp8dt = spr[15] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[13] * up)
      
  dm8dt = spr[16] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[14] * um)
  dp9dt = spr[17] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - up) + (spr[15] * up)
      
  dm9dt = spr[18] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[16] * um)

  drpdt = spr[19] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) - D) + (spr[17] * up)
    
  drmdt = spr[20] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) - D) + (spr[18] * um)
    
  dspmdt = [dsdt, dp1dt, dm1dt, dp2dt, dm2dt, dp3dt, dm3dt, dp4dt, dm4dt,
            dp5dt, dm5dt, dp6dt, dm6dt, dp7dt, dm7dt, dp8dt, dm8dt,
            dp9dt, dm9dt, drpdt, drmdt]
  return dspmdt

# initial conditions
spr0 = (S, p1, m1, p2, m2, p3, m3, p4, m4,
        p5, m5, p6, m6, p7, m7, p8, m8,
        p9, m9, rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df10 = pd.DataFrame(s_p_r, columns=['Sensitive', 
                                  'Persister1', 'Mutator1', 
                                  'Persister2', 'Mutator2', 
                                  'Persister3', 'Mutator3', 
                                  'Persister4', 'Mutator4',
                                  'Persister5', 'Mutator5', 
                                  'Persister6', 'Mutator6', 
                                  'Persister7', 'Mutator7', 
                                  'Persister8', 'Mutator8', 
                                  'Persister9', 'Mutator9',  
                                  'Resistant Persister', 'Resistant Mutator'])
df10['Time'] = range(0, 100)
df10['Total Cells'] = (df10['Sensitive'] + 
                     df10['Persister1'] + df10['Mutator1'] + 
                     df10['Persister2'] + df10['Mutator2'] + 
                     df10['Persister3'] + df10['Mutator3'] + 
                     df10['Persister4'] + df10['Mutator4'] + 
                     df10['Persister5'] + df10['Mutator5'] + 
                     df10['Persister6'] + df10['Mutator6'] + 
                     df10['Persister7'] + df10['Mutator7'] + 
                     df10['Persister8'] + df10['Mutator8'] + 
                     df10['Persister9'] + df10['Mutator9'] + 
                     df10['Resistant Persister'] + df10['Resistant Mutator'])
df10['Resistant'] = (df10['Resistant Persister'] + df10['Resistant Mutator'])
df10['Sensitive Fraction'] = df10['Sensitive']/df10['Total Cells']
df10['Persister Fraction'] = (df10['Persister1'] + df10['Persister2'] + 
                            df10['Persister3']+ df10['Persister4'] +
                            df10['Persister5']+ df10['Persister6'] +
                            df10['Persister7']+ df10['Persister7'] +
                            df10['Persister9']+ df10['Resistant Persister'])/df10['Total Cells']
df10['Mutator Fraction'] = (df10['Mutator1'] + df10['Mutator2'] + 
                          df10['Mutator3'] + df10['Mutator4'] +
                          df10['Mutator5'] + df10['Mutator6'] +
                          df10['Mutator7'] + df10['Mutator8'] +
                          df10['Mutator9'] + df10['Resistant Mutator'])/df10['Total Cells']
df10['Resistant Fraction'] = df10['Resistant']/df10['Total Cells']

In [ ]:
# Fifteen steps
# time points
t = np.linspace(0, 100, num = 100)
K = 10000
# N is total population
S = 9000
p1 = 900
m1 = 100
p2 = 0
m2 = 0
p3 = 0
m3 = 0
p4 = 0
m4 = 0
p5 = 0
m5 = 0
p6 = 0
m6 = 0
p7 = 0
m7 = 0
p8 = 0
m8 = 0
p9 = 0
m9 = 0
p10 = 0
m10 = 0
p11 = 0
m11 = 0
p12 = 0
m12 = 0
p13 = 0
m13 = 0
p14 = 0
m14 = 0
rp = 0
rm = 0
Gs = 0.0
D = 0.25
up = (3e-3) * 14
um = up * 2
A = 14 
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K) - D)
  dp1dt = spr[1] * (  
      (((n-14)/A * fR + (A-(n-14))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up
      )
  dm1dt = spr[2] * (  
      (((n-14)/A * fR + (A-(n-14))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um
      )
  dp2dt = spr[3] * (  
      (((n-13)/A * fR + (A-(n-13))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[1] * up)
      
  dm2dt = spr[4] * (  
      (((n-13)/A * fR + (A-(n-13))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[2] * um)

  dp3dt = spr[5] * (  
      (((n-12)/A * fR + (A-(n-12))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[3] * up)
      
  dm3dt = spr[6] * (  
      (((n-12)/A * fR + (A-(n-12))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[4] * um)

  dp4dt = spr[7] * (  
      (((n-10)/A * fR + (A-(n-10))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[5] * up)

  dm4dt = spr[8] * (  
      (((n-10)/A * fR + (A-(n-10))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[6] * up)
      
  dp5dt = spr[9] * (  
      (((n-9)/A * fR + (A-(n-9))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[7] * up)
      
  dm5dt = spr[10] * (  
      (((n-9)/A * fR + (A-(n-9))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[8] * um)

  dp6dt = spr[11] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[9] * up)

  dm6dt = spr[12] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[10] * um)

  dp7dt = spr[13] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[11] * up)
      
  dm7dt = spr[14] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[12] * um)

  dp8dt = spr[15] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[13] * up)
      
  dm8dt = spr[16] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[14] * um)
  dp9dt = spr[17] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[15] * up)
      
  dm9dt = spr[18] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[16] * um)

  dp10dt = spr[19] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[17] * up)
      
  dm10dt = spr[20] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[18] * um)

  dp11dt = spr[21] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[19] * up)
      
  dm11dt = spr[22] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[20] * um)

  dp12dt = spr[23] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[21] * up)
      
  dm12dt = spr[24] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[22] * um)

  dp13dt = spr[25] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[23] * up)
      
  dm13dt = spr[26] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[24] * um)

  dp14dt = spr[27] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[25] * up)
      
  dm14dt = spr[28] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[26] * um)

  drpdt = spr[29] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) - D) + (spr[27] * up)
    
  drmdt = spr[30] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) - D) + (spr[28] * um)
    
  dspmdt = [dsdt, dp1dt, dm1dt, dp2dt, dm2dt, dp3dt, dm3dt, dp4dt, dm4dt,
            dp5dt, dm5dt, dp6dt, dm6dt, dp7dt, dm7dt, dp8dt, dm8dt,
            dp9dt, dm9dt, dp10dt, dm10dt, dp11dt, dm11dt, dp12dt, dm12dt,
            dp13dt, dm13dt, dp14dt, dm14dt, drpdt, drmdt]
  return dspmdt

# initial conditions
spr0 = (S, p1, m1, p2, m2, p3, m3, p4, m4,
        p5, m5, p6, m6, p7, m7, p8, m8,
        p9, m9, p10, m10, p11, m11, 
        p12, m12, p13, m13, p14, m14,   
        rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df15 = pd.DataFrame(s_p_r, columns=['Sensitive', 
                                  'Persister1', 'Mutator1', 
                                  'Persister2', 'Mutator2', 
                                  'Persister3', 'Mutator3', 
                                  'Persister4', 'Mutator4',
                                  'Persister5', 'Mutator5', 
                                  'Persister6', 'Mutator6', 
                                  'Persister7', 'Mutator7', 
                                  'Persister8', 'Mutator8', 
                                  'Persister9', 'Mutator9',
                                  'Persister10', 'Mutator10', 
                                  'Persister11', 'Mutator11', 
                                  'Persister12', 'Mutator12', 
                                  'Persister13', 'Mutator13', 
                                  'Persister14', 'Mutator14',   
                                  'Resistant Persister', 'Resistant Mutator'])
df15['Time'] = range(0, 100)
df15['Total Cells'] = (df15['Sensitive'] + 
                     df15['Persister1'] + df15['Mutator1'] + 
                     df15['Persister2'] + df15['Mutator2'] + 
                     df15['Persister3'] + df15['Mutator3'] + 
                     df15['Persister4'] + df15['Mutator4'] + 
                     df15['Persister5'] + df15['Mutator5'] + 
                     df15['Persister6'] + df15['Mutator6'] + 
                     df15['Persister7'] + df15['Mutator7'] + 
                     df15['Persister8'] + df15['Mutator8'] + 
                     df15['Persister9'] + df15['Mutator9'] +
                     df15['Persister10'] + df15['Mutator10'] +
                     df15['Persister11'] + df15['Mutator11'] + 
                     df15['Persister12'] + df15['Mutator12'] + 
                     df15['Persister13'] + df15['Mutator13'] + 
                     df15['Persister14'] + df15['Mutator14'] +   
                     df15['Resistant Persister'] + df15['Resistant Mutator'])
df15['Resistant'] = (df15['Resistant Persister'] + df15['Resistant Mutator'])
df15['Sensitive Fraction'] = df15['Sensitive']/df15['Total Cells']
df15['Persister Fraction'] = (df15['Persister1'] + df15['Persister2'] + 
                            df15['Persister3']+ df15['Persister4'] +
                            df15['Persister5']+ df15['Persister6'] +
                            df15['Persister7']+ df15['Persister7'] +
                            df15['Persister9']+ df15['Persister10']+
                            df15['Persister11']+ df15['Persister12']+
                            df15['Persister13']+ df15['Persister14']+
                            df15['Resistant Persister'])/df15['Total Cells']
df15['Mutator Fraction'] = (df15['Mutator1'] + df15['Mutator2'] + 
                          df15['Mutator3'] + df15['Mutator4'] +
                          df15['Mutator5'] + df15['Mutator6'] +
                          df15['Mutator7'] + df15['Mutator8'] +
                          df15['Mutator9'] + df15['Mutator10'] +
                          df15['Mutator11'] + df15['Mutator12'] +
                          df15['Mutator13'] + df15['Mutator14'] +
                          df15['Resistant Mutator'])/df15['Total Cells']
df15['Resistant Fraction'] = df15['Resistant']/df15['Total Cells']

In [ ]:
# Twenty steps
# time points
t = np.linspace(0, 100, num = 100)
K = 10000
# N is total population
S = 9000
p1 = 900
m1 = 100
p2 = 0
m2 = 0
p3 = 0
m3 = 0
p4 = 0
m4 = 0
p5 = 0
m5 = 0
p6 = 0
m6 = 0
p7 = 0
m7 = 0
p8 = 0
m8 = 0
p9 = 0
m9 = 0
p10 = 0
m10 = 0
p11 = 0
m11 = 0
p12 = 0
m12 = 0
p13 = 0
m13 = 0
p14 = 0
m14 = 0
p15 = 0
m15 = 0
p16 = 0
m16 = 0
p17 = 0
m17 = 0
p18 = 0
m18 = 0
p19 = 0
m19 = 0
rp = 0
rm = 0
Gs = 0.0
D = 0.25
up = (3e-3) * 19
um = up * 2
A = 19 
n = A-1
fR = 1.0

# differential equatinons
def diff(spr, t):
  dsdt = spr[0] * (Gs * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K) - D)
  dp1dt = spr[1] * (  
      (((n-18)/A * fR + (A-(n-18))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up
      )
  dm1dt = spr[2] * (  
      (((n-18)/A * fR + (A-(n-18))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um
      )
  dp2dt = spr[3] * (  
      (((n-17)/A * fR + (A-(n-17))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[1] * up)
      
  dm2dt = spr[4] * (  
      (((n-17)/A * fR + (A-(n-17))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[2] * um)

  dp3dt = spr[5] * (  
      (((n-16)/A * fR + (A-(n-16))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[3] * up)
      
  dm3dt = spr[6] * (  
      (((n-16)/A * fR + (A-(n-16))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[4] * um)

  dp4dt = spr[7] * (  
      (((n-15)/A * fR + (A-(n-15))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[5] * up)

  dm4dt = spr[8] * (  
      (((n-15)/A * fR + (A-(n-15))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[6] * up)
      
  dp5dt = spr[9] * (  
      (((n-14)/A * fR + (A-(n-14))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[7] * up)
      
  dm5dt = spr[10] * (  
      (((n-14)/A * fR + (A-(n-14))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[8] * um)

  dp6dt = spr[11] * (  
      (((n-13)/A * fR + (A-(n-13))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[9] * up)

  dm6dt = spr[12] * (  
      (((n-13)/A * fR + (A-(n-13))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[10] * um)

  dp7dt = spr[13] * (  
      (((n-12)/A * fR + (A-(n-12))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[11] * up)
      
  dm7dt = spr[14] * (  
      (((n-12)/A * fR + (A-(n-12))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[12] * um)

  dp8dt = spr[15] * (  
      (((n-11)/A * fR + (A-(n-11))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[13] * up)
      
  dm8dt = spr[16] * (  
      (((n-11)/A * fR + (A-(n-11))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[14] * um)
  dp9dt = spr[17] * (  
      (((n-10)/A * fR + (A-(n-10))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[15] * up)
      
  dm9dt = spr[18] * (  
      (((n-10)/A * fR + (A-(n-10))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[16] * um)

  dp10dt = spr[19] * (  
      (((n-9)/A * fR + (A-(n-9))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[17] * up)
      
  dm10dt = spr[20] * (  
      (((n-9)/A * fR + (A-(n-9))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[18] * um)

  dp11dt = spr[21] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[19] * up)
      
  dm11dt = spr[22] * (  
      (((n-8)/A * fR + (A-(n-8))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[20] * um)

  dp12dt = spr[23] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[21] * up)
      
  dm12dt = spr[24] * (  
      (((n-7)/A * fR + (A-(n-7))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20]
                                )/K)) 
      - D - um) + (spr[22] * um)

  dp13dt = spr[25] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[23] * up)
      
  dm13dt = spr[26] * (  
      (((n-6)/A * fR + (A-(n-6))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[24] * um)

  dp14dt = spr[27] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[25] * up)
      
  dm14dt = spr[28] * (  
      (((n-5)/A * fR + (A-(n-5))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[26] * um)

  dp15dt = spr[29] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[27] * up)
      
  dm15dt = spr[30] * (  
      (((n-4)/A * fR + (A-(n-4))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[28] * um)

  dp16dt = spr[31] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[29] * up)
      
  dm16dt = spr[32] * (  
      (((n-3)/A * fR + (A-(n-3))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[30] * um)

  dp17dt = spr[33] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[31] * up)
      
  dm17dt = spr[34] * (  
      (((n-2)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[32] * um)

  dp18dt = spr[35] * (  
      (((n-1)/A * fR + (A-(n-2))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[33] * up)
      
  dm18dt = spr[36] * (  
      (((n-1)/A * fR + (A-(n-1))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[34] * um)

  dp19dt = spr[37] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - up) + (spr[33] * up)
      
  dm19dt = spr[38] * (  
      (((n)/A * fR + (A-(n))/A * D) 
      * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) 
      - D - um) + (spr[36] * um)

  drpdt = spr[39] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) - D) + (spr[37] * up)
    
  drmdt = spr[40] * ((fR * (1 - (spr[0] + spr[1] + spr[2] + spr[3] + spr[4] + spr[5] + spr[6] + spr[7] + spr[8] + spr[9] + spr[10] + 
                                spr[11] + spr[12] + spr[13] + spr[14] + spr[15] + spr[16] + spr[17] + spr[18] + spr[19] + spr[20] +
                                spr[21] + spr[22] + spr[23] + spr[24] + spr[25] + spr[26] + spr[27] + spr[28] + spr[29] + spr[30] 
                                )/K)) - D) + (spr[38] * um)
    
  dspmdt = [dsdt, dp1dt, dm1dt, dp2dt, dm2dt, dp3dt, dm3dt, dp4dt, dm4dt,
            dp5dt, dm5dt, dp6dt, dm6dt, dp7dt, dm7dt, dp8dt, dm8dt,
            dp9dt, dm9dt, dp10dt, dm10dt, dp11dt, dm11dt, dp12dt, dm12dt,
            dp13dt, dm13dt, dp14dt, dm14dt, dp15dt, dm15dt, dp16dt, dm16dt,
            dp17dt, dm17dt, dp18dt, dm18dt, dp19dt, dm19dt, drpdt, drmdt]
  return dspmdt

# initial conditions
spr0 = (S, p1, m1, p2, m2, p3, m3, p4, m4,
        p5, m5, p6, m6, p7, m7, p8, m8,
        p9, m9, p10, m10, p11, m11, 
        p12, m12, p13, m13, p14, m14,
        p15, m15, p16, m16, p17, m17,
        p18, m18, p19, m19, rp, rm)

# solve ODE
# the parameters are, the equations, initial conditions, 
# and time steps (between 0 and 100)
s_p_r = odeint(diff, spr0, t)
df20 = pd.DataFrame(s_p_r, columns=['Sensitive', 
                                  'Persister1', 'Mutator1', 
                                  'Persister2', 'Mutator2', 
                                  'Persister3', 'Mutator3', 
                                  'Persister4', 'Mutator4',
                                  'Persister5', 'Mutator5', 
                                  'Persister6', 'Mutator6', 
                                  'Persister7', 'Mutator7', 
                                  'Persister8', 'Mutator8', 
                                  'Persister9', 'Mutator9',
                                  'Persister10', 'Mutator10', 
                                  'Persister11', 'Mutator11', 
                                  'Persister12', 'Mutator12', 
                                  'Persister13', 'Mutator13', 
                                  'Persister14', 'Mutator14',
                                  'Persister15', 'Mutator15', 
                                  'Persister16', 'Mutator16', 
                                  'Persister17', 'Mutator17', 
                                  'Persister18', 'Mutator18', 
                                  'Persister19', 'Mutator19',    
                                  'Resistant Persister', 'Resistant Mutator'])
df20['Time'] = range(0, 100)
df20['Total Cells'] = (df20['Sensitive'] + 
                     df20['Persister1'] + df20['Mutator1'] + 
                     df20['Persister2'] + df20['Mutator2'] + 
                     df20['Persister3'] + df20['Mutator3'] + 
                     df20['Persister4'] + df20['Mutator4'] + 
                     df20['Persister5'] + df20['Mutator5'] + 
                     df20['Persister6'] + df20['Mutator6'] + 
                     df20['Persister7'] + df20['Mutator7'] + 
                     df20['Persister8'] + df20['Mutator8'] + 
                     df20['Persister9'] + df20['Mutator9'] +
                     df20['Persister10'] + df20['Mutator10'] +
                     df20['Persister11'] + df20['Mutator11'] + 
                     df20['Persister12'] + df20['Mutator12'] + 
                     df20['Persister13'] + df20['Mutator13'] + 
                     df20['Persister14'] + df20['Mutator14'] +
                     df20['Persister15'] + df20['Mutator15'] + 
                     df20['Persister16'] + df20['Mutator16'] + 
                     df20['Persister17'] + df20['Mutator17'] + 
                     df20['Persister18'] + df20['Mutator18'] + 
                     df20['Persister19'] + df20['Mutator19'] +    
                     df20['Resistant Persister'] + df20['Resistant Mutator'])
df20['Resistant'] = (df20['Resistant Persister'] + df20['Resistant Mutator'])
df20['Sensitive Fraction'] = df20['Sensitive']/df20['Total Cells']
df20['Persister Fraction'] = (df20['Persister1'] + df20['Persister2'] + 
                            df20['Persister3']+ df20['Persister4'] +
                            df20['Persister5']+ df20['Persister6'] +
                            df20['Persister7']+ df20['Persister7'] +
                            df20['Persister9']+ df20['Persister10']+
                            df20['Persister11']+ df20['Persister12']+
                            df20['Persister13']+ df20['Persister14']+
                            df20['Persister15']+ df20['Persister16']+
                            df20['Persister18']+ df20['Persister18']+
                            df20['Persister19']+ df20['Resistant Persister'])/df20['Total Cells']
df20['Mutator Fraction'] = (df20['Mutator1'] + df20['Mutator2'] + 
                          df20['Mutator3'] + df20['Mutator4'] +
                          df20['Mutator5'] + df20['Mutator6'] +
                          df20['Mutator7'] + df20['Mutator8'] +
                          df20['Mutator9'] + df20['Mutator10'] +
                          df20['Mutator11'] + df20['Mutator12'] +
                          df20['Mutator13'] + df20['Mutator14'] +
                          df20['Mutator15'] + df20['Mutator16'] +
                          df20['Mutator17'] + df20['Mutator18'] +
                          df20['Mutator19'] + df20['Resistant Mutator'])/df20['Total Cells']
df20['Resistant Fraction'] = df20['Resistant']/df20['Total Cells']